<a href="https://colab.research.google.com/github/ankitk75/Big-Data-Analytics-Lab/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import pandas as pd

# Define the input and output file paths
input_file_path = '/content/input.txt'
output_file_path = '/content/output.csv'

# Initialize lists to hold questions and answers
questions = []
answers = []

# Read the input file and parse the content
with open(input_file_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.strip():  # Skip any empty lines
            question, answer = line.split(',', 1)  # Split only at the first comma
            questions.append(question.strip())
            answers.append(answer.strip())

# Write the questions and answers to a new CSV file
with open(output_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['question', 'answer'])  # Write the header row
    for question, answer in zip(questions, answers):
        csvwriter.writerow([question, answer])


In [2]:
def load_dataset_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    dialogues = [{"input": row['question'], "output": row['answer']} for _, row in df.iterrows()]
    return Dataset.from_pandas(pd.DataFrame(dialogues))


# dataset = load_dataset_from_csv('output.csv')

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Add padding token
tokenizer.pad_token = tokenizer.eos_token

# Prepare the dataset
dataset = load_dataset_from_csv('output.csv')
train_test_split = dataset.train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})


# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['input'], truncation=True, padding='max_length', max_length=128)
    outputs = tokenizer(examples['output'], truncation=True, padding='max_length', max_length=128)
    return {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask'],
            'labels': outputs['input_ids']}


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [8]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [11]:
!pip install accelerate -U

In [5]:
# Tokenize the dataset
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,8.505600,8.867254
2,7.044500,4.484799
3,4.412800,3.542815


TrainOutput(global_step=36, training_loss=6.096654733022054, metrics={'train_runtime': 971.272, 'train_samples_per_second': 0.139, 'train_steps_per_second': 0.037, 'total_flos': 31343648440320.0, 'train_loss': 6.096654733022054, 'epoch': 3.0})

In [6]:
model.save_pretrained("fine-tuned-dialoGPT-medium")
tokenizer.save_pretrained("fine-tuned-dialoGPT-medium")

('fine-tuned-dialoGPT-medium/tokenizer_config.json',
 'fine-tuned-dialoGPT-medium/special_tokens_map.json',
 'fine-tuned-dialoGPT-medium/vocab.json',
 'fine-tuned-dialoGPT-medium/merges.txt',
 'fine-tuned-dialoGPT-medium/added_tokens.json',
 'fine-tuned-dialoGPT-medium/tokenizer.json')

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("fine-tuned-dialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("fine-tuned-dialoGPT-medium")

def run(user_text, chat_history_ids):
    input_ids = tokenizer.encode(user_text + tokenizer.eos_token, return_tensors='pt')
    if chat_history_ids is None:
        bot_history_ids = input_ids
    else:
        bot_history_ids = torch.cat([chat_history_ids, input_ids], dim=-1)

    chat_history_ids = model.generate(bot_history_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    resp = tokenizer.decode(chat_history_ids[:, bot_history_ids.shape[-1]:][0], skip_special_tokens=True)

    return resp, chat_history_ids


In [10]:
from google.colab import files

In [14]:
# files.download("fine-tuned-dialoGPT-medium")

In [16]:
import shutil

shutil.make_archive('fine-tuned-dialoGPT-medium', 'zip', 'fine-tuned-dialoGPT-medium')  # Replace 'folder_name' with your actual folder name


'/content/fine-tuned-dialoGPT-medium.zip'

In [17]:
!pip install streamlit streamlit-chat numpy  # Install required libraries (commented out for Colab execution)

import streamlit as st
from streamlit_chat import message as st_message
import numpy as np

def run(txt, chat_history_ids=None):
  # Replace this function with your actual "run" function
  # that processes the user input and returns the response and history.
  # For example, it could call a chatbot model.
  return f"Thanks for your message: {txt}", chat_history_ids

if "chat_history_ids" not in st.session_state:
  st.session_state["chat_history_ids"] = None
if "book" not in st.session_state:
  st.session_state["book"] = []

text = st.text_input("Type Here")

if text:
  response, history = run(text, st.session_state["chat_history_ids"])
  st.session_state["chat_history_ids"] = history
  st.session_state["book"].append({"message": text, "is_user": True})
  st.session_state["book"].append({"message": response, "is_user": False})

for i, chat in enumerate(st.session_state["book"]):
  st_message(**chat, key=str(i))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


2024-05-28 12:41:45.834 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-05-28 12:41:45.939 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.188.159:8501

